In [2]:
from transformers import AutoTokenizer
import transformers
import torch
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import langchain

langchain.debug = True

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"
 
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # eturn_full_text=True,
    torch_dtype=torch.float16,
    device_map="auto",
    # we pass model parameters here too
    temperature=0.6,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
    )

llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

In [5]:
# initialize agent
agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory
)

In [6]:
from prompts import *
from prompts import calculator

In [7]:
sys_msg = "<s>" + B_SYS + calculator.SYSTEM_MESSAGE + E_SYS
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = new_prompt
instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values " + E_INST
human_msg = instruction + "\nUser: {input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [8]:
# llm(prompt=sys_msg + human_msg.format(input="what's 99+100*11 ?"))
agent("what's 99+100*11 ?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what's 99+100*11 ?",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "what's 99+100*11 ?",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "System: <s><<SYS>>\nAssistant is a expert JSON builder designed to assist with a wide range of tasks.\n\nAssistant is able to respond to the User and use tools using JSON strings that contain \"action\" and \"action_input\" parameters.\n\nAll of Assistant's communication is performed using this JSON format.\n\nAssistant can also use tools by responding to the user with tool use instructions in the same \"action\" and \"action_input\" JSON format. Tools available to Assistant are:\n\n- \"Calculator\":

OutputParserException: Could not parse LLM output: 